In [1]:
# Libraries
from sqlalchemy import create_engine
import pandas as pd
import seaborn as sns

## Connect Database

In [2]:
# Note:: The make sure you use the information from your specific PostgreSQL installation
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'11207471' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'cleaned' # schema we just created

In [3]:
db_conn = create_engine(f"postgresql://{user}:{pw}@{host}:{port}/{db}")

In [4]:
table_name = r'global_warming2'

In [5]:
sql="select tables.table_name from information_schema.tables where (table_schema ='"+schema+"')order by 1;"
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name
0,global_warming2
1,global_warming_Wth_Risk_Level
2,global_warming_cleaned


## Select Data

In [6]:
sql=r'SELECT * FROM ' + schema + '.' + table_name
gb = pd.read_sql(sql, db_conn, index_col=None)

In [7]:
gb.head(10)

,Country,Year,Temperature_Anomaly,CO2_Emissions,Population,Forest_Area,GDP,Renewable_Energy_Usage,Methane_Emissions,Sea_Level_Rise,...,Per_Capita_Emissions,Industrial_Activity,Air_Pollution_Index,Biodiversity_Index,Ocean_Acidification,Fossil_Fuel_Usage,Energy_Consumption_Per_Capita,Policy_Score,Average_Temperature,Risk_Level
0,Country_1,1900,-0.736197,2.733557e+08,3.186188e+08,15.244316,5.450396e+12,53.313474,5.963399e+06,21.498814,...,7.583504,60.305802,119.849255,49.569774,8.006157,55.004982,2474.748082,53.882270,6.971064,0
1,Country_1,1901,-0.703154,6.998442e+08,1.494792e+09,54.731054,9.044301e+12,46.218131,2.359518e+06,6.527975,...,19.029111,33.844962,173.091676,9.140253,8.107997,56.227115,3411.432550,17.905168,32.393594,1
2,Country_1,1902,-0.269372,6.387650e+08,1.769422e+08,30.736100,3.577474e+12,81.932127,5.874300e+06,37.340943,...,5.724153,40.501150,163.346198,65.776186,7.923555,49.416237,1988.796272,53.597921,16.137289,2
3,Country_1,1903,1.138549,7.073459e+08,8.504665e+08,68.558736,3.423279e+12,30.812757,3.934211e+06,19.431326,...,6.383886,60.212029,170.589781,52.224952,7.887093,54.237972,2143.174604,35.826989,23.842348,2
4,Country_1,1904,0.272228,6.806187e+08,5.487387e+08,30.130873,3.364767e+12,42.098978,2.613544e+06,6.505570,...,2.562015,57.873658,110.710126,54.761718,7.730068,26.553637,4023.094770,43.621281,27.347198,2
5,Country_1,1905,0.779972,6.848920e+08,8.218035e+08,53.451252,5.989337e+12,71.220964,6.835859e+06,14.517514,...,11.344739,63.237688,125.723362,73.887508,7.807014,66.237886,2906.285255,50.310318,12.986597,0
6,Country_1,1906,0.749481,7.384828e+08,9.717844e+08,71.304433,6.368574e+12,40.377358,4.102873e+06,8.631829,...,11.720899,80.471674,129.663990,72.375680,7.897264,70.859571,1393.793605,57.001642,3.349868,1
7,Country_1,1907,-0.637168,6.896140e+08,8.765094e+08,60.596205,7.779992e+12,50.787751,2.742369e+06,31.410363,...,12.749148,50.260546,51.202577,25.957086,8.057036,42.843479,3123.596058,46.781841,34.239512,1
8,Country_1,1908,0.273943,3.965448e+08,9.029806e+08,35.219464,6.451774e+12,39.634114,2.796150e+06,29.593069,...,9.452678,31.780187,200.250576,68.295645,7.941333,52.268319,2860.103473,51.230245,14.993037,1
9,Country_1,1909,0.816333,3.118423e+08,1.311659e+09,27.012174,8.802959e+12,37.756790,1.785549e+06,48.850478,...,11.548370,26.583421,48.526769,28.153486,8.138102,43.206151,3803.911623,28.900588,28.454129,1


## Model

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# Random Forest model

X = gb[['CO2_Emissions', 'Methane_Emissions','GDP']]

y = gb['Risk_Level'] # Dependent


# Split data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_hold, y_val, y_hold = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)


# Print dataset sizes
print("Training set:", X_train.shape)
print("Validation set:", X_val.shape)
print("Test set:", X_test.shape)

# Intitalize Rnd

rnd2 = RandomForestClassifier(n_estimators=100, random_state=42)


# Model Train

rnd2.fit(X_train, y_train)

Training set: (16657, 3)
Validation set: (3570, 3)
Test set: (7140, 3)


RandomForestClassifier(random_state=42)

In [10]:

predictions = rnd2.predict(X_val)
predictions

array([0, 1, 1, ..., 2, 1, 0])

In [12]:
predictions.shape

(3570,)

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Calculate Accuracy
accuracy = accuracy_score(y_val, predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [22]:
# convert prediction to a dataframe
predictions_df = pd.DataFrame(predictions, columns=['Predicted_Risk_Level'])


## Save prediction into a nw schema

In [23]:
schema = r'analytics' # schema we just created
sql="select tables.table_name from information_schema.tables where (table_schema ='"+schema+"')order by 1;"
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name


In [24]:
table_name = r'model1_validation_predict'

In [25]:
predictions_df.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')


3570

In [26]:
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name
0,model1_validation_predict


In [27]:
sql=r'SELECT * FROM ' + schema + '.' + table_name
clean_check = pd.read_sql(sql, db_conn, index_col=None)

In [28]:
clean_check.head(10)

,Predicted_Risk_Level
0,0
1,1
2,1
3,2
4,2
5,2
6,1
7,1
8,0
9,0


## Summary

In notebooks Week6_Lab_part1 and Week6_Lab_part2, we examine constructing a model using features from last week’s lab and selecting new features as well.

In the first notebook (Week6_Lab_part1), I created two models.

Before constructing the models, I modified my KMeans model. In last week’s lab, the KMeans clustering used the features I would later use for my random forest classification model to create clusters. However, this time I used all the columns in my dataset as features in the KMeans model so that the clusters can identify patterns across the entire dataset.

Once I obtained a risk level output from the KMeans, I proceeded to create my first random forest classification model. For this model, the input features were all the columns in my dataset except for the risk level column, which served as my target. My goal in using all the columns as features was to uncover another important feature for the model, beyond those identified last week.

After running the model, I found that GDP was the most important feature; all other columns/features were less significant.

In my second model, I selected the two features from last week along with the newly selected feature, GDP. The features used were:
`gb[['CO2_Emissions', 'Methane_Emissions', 'GDP']]`.

After running the model, I achieved 100% accuracy for model 2, just as with model 1. When comparing feature importance in model 2, the importance of my selected features increased.

In the second notebook, I evaluated model 2 in a manner similar to notebook 1 and generated predictions for my validation set. For the validation set, 3570 datasets were allocated. After generating predictions, we checked the accuracy using accuracy_score. With a value of 1.0, we know our model predicts 100% accurately.